https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [166]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn import preprocessing

In [167]:
data = pd.read_csv('train.csv')
practice = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [168]:
label = data.pop('SalePrice')

In [169]:
def pre_process(df_train, df_test):
    # Convert all df to str
    df_train = df_train.applymap(lambda x: str(x))
    df_test = df_test.applymap(lambda x: str(x))
    # replace NaN by 0
    df_train.fillna('0', inplace=True) # if fill with int(0), label encoder fails
    df_test.fillna('0', inplace=True)
    # Drop Id
    df_train.drop('Id', axis=1, inplace=True)
    df_test.drop('Id', axis=1, inplace=True)
    # Label encode each non numerical features
    dict_encoder = {}
    for col in list(df_train.select_dtypes(include='object')):
        le = preprocessing.LabelEncoder()
        le.fit(list(df_train[col])+list(df_test[col]))
        df_train[col] = le.transform(df_train[col])
        df_test[col] = le.transform(df_test[col])
        dict_encoder[col] = le
    # Re-convert all df to float
    df_train = df_train.applymap(lambda x: float(x))
    df_test = df_test.applymap(lambda x: float(x))
    return df_train, df_test, dict_encoder

In [170]:
data, practice, dict_encoder = pre_process(data, practice)